# How-to: create a WNTR WaterNetworkModel from scratch

This notebook demonstrates how to create a 
WaterNetworkModel completely from scratch. It uses the Net3
example that comes from EPANET to do this, as it is sufficiently
interesting but not super complicated. After creating the model,
the handmade model will be simulated and the results compared
to results from loading in the original INP file and running 
that model.

This file was created partially by hand and partially by using a 
spreadsheet to take an existing EPANET INP file (Net3.inp) and 
using string concatenation functions to create the appropriate
Python calls. It is possible to write this file by hand, obviously,
but since the INP file exists, it is much faster to do it using
text formatting.

#### Table of Contents

## Setup
If you have not installed WNTR yet, you need to do so. WNTR
uses additional libraries, though if you use an anaconda or
Enthought distribution, you will probably have most of them
installed. If you do "pip install wntr", it should verify
that you have all the dependencies.

For this demo, we need WNTR and for graphics we will check to make sure plotly and matplotlib are also installed.

In [ ]:
import wntr
import matplotlib.pyplot as plt
import plotly as ptly

## Building the model
The first thing to do is create a new, empty WaterNetworkModel. We will then set up some handy shortcuts to cut down on how much typing we need to do.

Choose the units that will be used in conversions and for the EPANET INP file (if the EpanetSimulator is used).
Remember that WNTR uses SI units internally (kg, m, s, Pa, etc.) for all properties. Since we are 
replicating the EPANET Net3 example, we will use US Customary units of Gallons/minute as the flow units and
convert properties like elevations to SI units as we go.

Here we create some shortcuts so that it is easier to read and write the commands.

In [ ]:
wn = wntr.network.model.WaterNetworkModel()  # Create a new water network model
wn.name = 'MyNewNet3'
opts = wn.options                            # Shortcut to the water network options structure
toSI = wntr.epanet.util.to_si                # Shortcut to the USC to SI units conversion function
hydPar = wntr.epanet.util.HydParam           # Shortcut to the hydraulic parameter types
qualPar = wntr.epanet.util.QualParam         # Shortcut to the quality parameter types

### Setting the model's options
The WaterNetworkOptions class incorporates the EPANET sections for 
Options, Times, Quality, and Energy for setting up the model and 
simulators. It also stores EPANET graphics and outputs options from 
Report, Map, Backdrop, and Vertices. The core options will be discussed
next, and the graphics and output options will be discussed later.

#### Hydraulic options
The options for the model need to be specified first, before anything else. 
This includes setting the flow units, default pattern name (if desired),
solver options, and quality mode. All these options (except default pattern name)
can be changed later. All these options come from the EPANET [OPTIONS] section.

In [ ]:
# Set up the EPANET flow units
flowunits = wntr.epanet.util.FlowUnits.GPM   # Choose GPM as EPANET's units
massunits = wntr.epanet.util.MassUnits.mg    # Choose quality in mg/L
opts.hydraulic.units = flowunits

# Choose a headloss formula to use
headlosseq = wntr.epanet.util.FormulaType.HW # HEADLOSS H-W 
opts.hydraulic.headloss = headlosseq        

opts.hydraulic.viscosity = 1.0
opts.hydraulic.specific_gravity = 1.0
opts.hydraulic.emitter_exponent = toSI(flowunits, 0.5, hydPar.EmitterCoeff)
opts.hydraulic.pattern = '1'

The emitter exponent is the only hydraulic option that needs unit conversion.
#### Solver options
Setting the options for the solver is important. The following are options that
apply to the EpanetSimulator, and all come from the [OPTIONS] section in an
EPANET inp file.

In [ ]:
opts.solver.trials = 40
opts.solver.accuracy = 0.001
opts.solver.checkfreq = 2
opts.solver.unbalanced = 'CONTINUE'
opts.solver.unbalanced_value = 10
opts.solver.tolerance = 0.01

#### Time options
The times for a WNTR model must be specified in seconds. All the timing options come from EPANET's [TIMES] section.

In [ ]:
opts.time.duration = 168 * 60 * 60          # 168:00:00
opts.time.hydraulic_timestep = 15 * 60      #     15:00
opts.time.quality_timestep = 15 * 60        #     15:00
opts.time.pattern_timestep = 60 * 60        #   1:00:00
opts.time.pattern_start = 0                 #   0:00:00
opts.time.report_timestep = 15 * 60         #     15:00
opts.time.report_start = 0                  #   0:00:00
opts.time.start_clocktime = 0               #  12:00:00 AM

#### Water quality options
The `quality` options come from two different places in the inp file, three from [OPTIONS] and the remainder from [REACTIONS].
The "quality" option in EPANET takes two values. 
These two values are separated in WNTR. The following sets up a water quality trace from the lake.
The water quality reaction options are stored in WNTR as per-second, not per-day, and must be converted. The values here are 0.0, but the conversion is shown anyway

In [ ]:
# From the [OPTIONS] section of an inp file
opts.quality.mode = 'TRACE'
opts.quality.trace_node = 'Lake'
opts.quality.diffusivity = 1.0

# From the [REACTIONS] section of an inp file
opts.quality.bulk_rxn_order = 0
opts.quality.tank_rxn_order = 0
opts.quality.wall_rxn_order = 0
opts.quality.limiting_potential = 0
opts.quality.roughness_correlation = 0
opts.quality.bulk_rxn_coeff = toSI(flowunits, 0.0, 
                                   param=qualPar.BulkReactionCoeff,
                                   mass_units=massunits,
                                   reaction_order=opts.quality.bulk_rxn_order)
opts.quality.wall_rxn_coeff = toSI(flowunits, 0.0, 
                                   param=qualPar.WallReactionCoeff,
                                   mass_units=massunits,
                                   reaction_order=opts.quality.wall_rxn_order)

#### Energy computation options
Pump energy consumption calculations use the `energy` section of the options structure. All the [ENERGY] parameters in an EPANET inp file are included here.

In [ ]:
opts.energy.demand_charge = 0
opts.energy.global_price = 0
opts.energy.global_efficiency = 75
opts.energy.global_pattern = None

#### Results options
The results options allow the user to get a single statistic as a result or to specify that only certain types of results should be saved after the simulation. 

In [ ]:
opts.results.statistic = 'NONE'

### Creating Patterns and Curves
Once the options have been set up, the next step is to create the patterns. Note that the order of sections doesn't matter in EPANET, as everything is read in first, then parsed. To create a network model programmatically, the patterns must exist in the network before assigning them to any objects (like junctions). 

#### Patterns
The simplest way to create a pattern is to pass a name and list of multipliers. Because patterns are unitless, there is no need to run any unit conversion. Use the water network model's `add_pattern` to add the pattern to the model.

In [ ]:
wn.add_pattern('1', [1.34, 1.94, 1.46, 1.44, 0.76, 0.92, 0.85, 1.07,
                     0.96, 1.1, 1.08, 1.19, 1.16, 1.08, 0.96, 0.83,
                     0.79, 0.74, 0.64, 0.64, 0.85, 0.96, 1.24, 1.67])
wn.add_pattern('2', [0, 0, 0, 0, 0, 1219, 0, 0,
                     0, 1866, 1836, 1818, 1818, 1822, 1822, 1817,
                     1824, 1816, 1833, 1817, 1830, 1814, 1840, 1859])
wn.add_pattern('3', [620, 620, 620, 620, 620, 360, 360, 0,
                     0, 0, 0, 360, 360, 360, 360, 360,
                     0, 0, 0, 0, 0, 0, 360, 360])

There is an additional way to create a pattern. It is possible to create a `wntr.network.Pattern` object first, and pass the object in.

In [ ]:
wn.add_pattern('4', [0])  # we will be changing this later
pat5 = wntr.network.Pattern('5', [4439, 4531, 4511, 4582, 4531, 4582, 4572, 4613,
                                  4643, 4643, 4592, 4613, 4531, 4521, 4449, 4439,
                                  4449, 4460, 4439, 4419, 4368, 4399, 4470, 4480])
wn.add_pattern('5', pat5)

To get a pattern, call the model's `get_pattern` method.

In [ ]:
pat4 = wn.get_pattern('4')

The pattern values can be modified by assigning new values to the `.multipliers` attribute. Once a pattern has been retreived through the `get_pattern` method, it is not necessary to re-add it to the model.

In [ ]:
print(str(pat4), 'Original multipliers: ', pat4.multipliers)
pat4mult = [1637, 1706, 1719, 1719, 1791, 1819, 1777, 1842,
            1815, 1825, 1856, 1801, 1819, 1733, 1664, 1620,
            1613, 1620, 1616, 1647, 1627, 1627, 1671, 1668]
pat4.multipliers = pat4mult
print('')
print(wn.get_pattern('4').multipliers)

Adding the pattern with the same name will result in an error.

In [ ]:
try:
    wn.add_pattern('4', pat4)
except Exception as e:
    print(repr(e))

#### Curves
Unlike patterns, Curve objects are specific to a type of curve. For example, volume curves and pump (head) curves have completely different units on their axes, so it makes no sense to try to use the same object for both. The two curves that are present in the Net3 model are both "pump curves," a.k.a., "head curves".

To create the curve, the X and Y values must be properly converted. Head curves use flow for the X axis and head for the Y axis. Notice that it is possible to convert arrays of values (or, in fact, dictionaries of values) all at once.
The x,y values must be passed in to the water network model as a list of (x,y) pairs. There is an easy way to do this in Python, using a function called `zip`.

In [ ]:
curve1x = toSI(flowunits, [0, 2000, 4000], hydPar.Flow)
curve1y = toSI(flowunits, [104, 92, 63], hydPar.HydraulicHead)
curve2x = toSI(flowunits, [0, 8000, 14000], hydPar.Flow)
curve2y = toSI(flowunits, [200, 138, 86], hydPar.HydraulicHead)
curve1_coeff = zip(curve1x, curve1y)
curve2_coeff = zip(curve2x, curve2y)

print(curve1x, curve1y)
print(curve1_coeff)

Once the coefficients have been created, the curve objects can be created and added to the model. We will use these curves when we add pumps later on.

In [ ]:
wn.add_curve(name='1', curve_type='HEAD', xy_tuples_list=curve1_coeff)
wn.add_curve(name='2', curve_type='HEAD', xy_tuples_list=curve2_coeff)

print(repr(wn.get_curve('2')))

### Adding Nodes and Links
Once the options and patterns have been configured, we can proceed to creating the physical elements, nodes and links.

#### Junctions
Because there are 97 nodes in this model, we will first add a few junctions, the tanks, and the reservoirs in order to demonstrate the syntax. Afterwards, the remaining 90 or so junctions will be added without extra explanations.
There are several ways to add junctions, based on the information that is available. Technically, only the name is a required option.

Example 1: For nodes with no demand, it is possible to add simply the elevation (converted from ft to SI units here). Here we also add coordinates to the node.

Example 2: For nodes with a base demand, it should be converted to SI units (here being converted from gallons per minute to m^3/second). A demand pattern should be specified by name if one is desired.

Example 3: *If the default pattern is* defined (`wn.options.hydraulic.pattern is not None`), then if the `demand_pattern` is not provided, the default pattern is used. *If the default pattern is None* then the base demand will be used as a constant demand value. If a default pattern is defined and a constant value is needed, create a pattern with a single multiplier of `[1.0]` and assign that pattern where constant demand is desired.

In [ ]:
# Example 1: adding a junction with no demand
wn.add_junction(name='10', 
                elevation=toSI(flowunits, 147, hydPar.Elevation), 
                coordinates=(9, 27.85))

# Example 2: adding a junction using explicit demand pattern
wn.add_junction(name='15', 
                elevation=toSI(flowunits, 32, hydPar.Elevation), 
                base_demand=toSI(flowunits, 1, hydPar.Demand), 
                demand_pattern='3', 
                coordinates=(38.68, 23.76))

# Example 3: adding a junction using the default demand pattern, set in options
wn.add_junction(name='101', 
                elevation=toSI(flowunits, 42, hydPar.Elevation), 
                base_demand=toSI(flowunits, 189.95, hydPar.Demand), 
                coordinates=(13.81, 22.94))

print(repr(wn.get_node('15')))

#### Tanks

In [ ]:
wn.add_tank(name='1', 
                elevation=toSI(flowunits, 131.9, hydPar.Elevation), 
                init_level=toSI(flowunits, 13.1, hydPar.Length), 
                min_level=toSI(flowunits, 0.1, hydPar.Length), 
                max_level=toSI(flowunits, 32.1, hydPar.Length), 
                diameter=toSI(flowunits, 85, hydPar.TankDiameter), 
                min_vol=toSI(flowunits, 0, hydPar.Volume), 
                coordinates=(27.46, 9.84))
wn.add_tank(name='2', 
                elevation=toSI(flowunits, 116.5, hydPar.Elevation), 
                init_level=toSI(flowunits, 23.5, hydPar.Length), 
                min_level=toSI(flowunits, 6.5, hydPar.Length), 
                max_level=toSI(flowunits, 40.3, hydPar.Length), 
                diameter=toSI(flowunits, 50, hydPar.TankDiameter), 
                min_vol=toSI(flowunits, 0, hydPar.Volume), 
                coordinates=(32.99, 3.45))
wn.add_tank(name='3', 
                elevation=toSI(flowunits, 129, hydPar.Elevation), 
                init_level=toSI(flowunits, 29, hydPar.Length), 
                min_level=toSI(flowunits, 4, hydPar.Length), 
                max_level=toSI(flowunits, 35.5, hydPar.Length), 
                diameter=toSI(flowunits, 164, hydPar.TankDiameter), 
                min_vol=toSI(flowunits, 0, hydPar.Volume), 
                coordinates=(29.41, 27.27))

print(repr(wn.get_node('3')))

#### Reservoirs

In [ ]:
wn.add_reservoir(name='Lake',
                 base_head=toSI(flowunits, 167, hydPar.HydraulicHead),
                 coordinates=(8, 27.53))
wn.add_reservoir(name='River',
                 base_head=toSI(flowunits, 220, hydPar.HydraulicHead),
                 coordinates=(24.15, 31.06))

print(repr(wn.get_node('Lake')))

#### Pipes
It is *not required* to add the appropriate nodes before adding the pipes. 
The following three junctions (60, 61, and 601) are defined so that the 
example definitions for pipes and pumps can be shown.

In [ ]:
# The following three junctions are used for defining the example links
wn.add_junction(name='60', elevation=toSI(flowunits, 0, hydPar.Elevation), coordinates=(23.9, 29.94))
wn.add_junction(name='61', elevation=toSI(flowunits, 0, hydPar.Elevation), coordinates=(23.71, 29.03))
wn.add_junction(name='601', elevation=toSI(flowunits, 0, hydPar.Elevation), coordinates=(23, 29.49))

wn.add_pipe(name='330', 
            start_node_name='60',
            end_node_name='601', 
            length=toSI(flowunits, 1, hydPar.Length), 
            diameter=toSI(flowunits, 30, hydPar.PipeDiameter), 
            roughness=toSI(flowunits, 140, hydPar.RoughnessCoeff), 
            minor_loss=0,
            status='Closed',  # Default is 'Open'
            check_valve_flag=False  # Default is False
            )

print(repr(wn.get_link('330')))

#### Pumps

In [ ]:
wn.add_pump(name='10',
            start_node_name='Lake',
            end_node_name='10',
            info_type='HEAD',
            info_value=wn.get_curve('1'),
            speed=1.0)
wn.add_pump(name='335',
            start_node_name='60',
            end_node_name='61',
            info_type='HEAD',
            info_value=wn.get_curve('2'))

print(repr(wn.get_link('335')))

#### Valves
The Net3 model does not have any valves. We will come back to valves 
when we start modifying the model

#### Initial link settings
Pipes can be opened, closed, or have a check valve assigned when they 
are created. However, for valves and pumps, it is only possible to 
set their initial status after they have been created.
Any link can have its initial status changed by using the `set_initial_status` 
member function. In Net3, pump '10' starts out as closed, and we set it
so here.

In [ ]:
pump10 = wn.get_link('10')
print('Current status: ',pump10.status)
pump10.set_initial_status('CLOSED')
print('New status: ',wn.get_link('10').status)

### Controls and Rules
In many ways, the rules (and controls) are the most important part of EPANET. These are the pieces that allow the user to truly mimic system operations, turning on pumps, shutting valves, and all the other actions that control the model. Unfortunately, this makes them more complex then the simple functions like `add_junction` or `add_pattern`. We will start with simple controls first, and then move on to more complex rules.

#### Simple time and comparison controls
The simplest controls are limited to a single change of a status or setting that is triggered by time, tank level or junction pressure. There are no if/then/else constructs, no multiple actions, and very basic comparisons. 

To start, we will create some references to the actions for opening and closing the pumps and pipes. We will also create some shortcuts to the classes we will need. Status values must be set using the `LinkStatus` enumerated class, not strings such as 'open' or 'closed'.

In [ ]:
ControlAction = wntr.network.ControlAction
closed = wntr.network.LinkStatus.Closed
opened = wntr.network.LinkStatus.Open
TimeControl = wntr.network.TimeControl
ConditionalControl = wntr.network.ConditionalControl

# remember that we created pump10 earlier when we set its initial status
act_close_pump10 = ControlAction(pump10, 'status', closed) 
act_open_pump10 = ControlAction(pump10, 'status', opened)
ctrl_t01 = TimeControl(wn,
                       run_at_time=1*3600,
                       time_flag='SIM_TIME',
                       daily_flag=False,
                       control_action=act_open_pump10)
ctrl_t15 = TimeControl(wn,
                       run_at_time=15*3600,
                       time_flag='SIM_TIME',
                       daily_flag=False,
                       control_action=act_close_pump10)

wn.add_control('Open_10At01', ctrl_t01)
wn.add_control('Close10At15', ctrl_t15)

Notice that there are several options that are *required* for creating a TimeControl. First, the model itself must be passed in to the constructor as the first argument. Second, `run_at_time` must be provided in sectonds. Third, the `time_flag` option can be either 'SIM_TIME' for simulation time or 'SHIFTED_TIME' for clock time. Fourth, the `daily_flag` option tells WNTR to execute the control every day at the same time -- *this is not a valid option if you want to use the EpanetSimulator*. Finally, the control option is passed in.

Next, we will create one of the level-based controls. We once again start with some variables and shortcuts.

In [ ]:
# Model objects and shortcuts
pump335 = wn.get_link('335')
pipe330 = wn.get_link('330')
tank1 = wn.get_node('1')
below = wntr.network.controls.Comparison.lt.func
above = wntr.network.controls.Comparison.gt.func

# New actions
act_close335 = ControlAction(pump335, 'status', closed)
act_open335 = ControlAction(pump335, 'status', opened)
act_close330 = ControlAction(pipe330, 'status', closed)
act_open330 = ControlAction(pipe330, 'status', opened)

# Thresholds to SI units
tanklow = toSI(flowunits, 17.1, hydPar.Length)
tankhi = toSI(flowunits, 19.1, hydPar.Length)

# Pump filling
# read the definition as:       IF  nodeY parameter above/below  threshold  thenDoSomeAction
ctrl_open335  = ConditionalControl((tank1, 'level'), below, tanklow, act_open335)
ctrl_close330 = ConditionalControl((tank1, 'level'), below, tanklow, act_close330)
# Pump draining 
ctrl_open330  = ConditionalControl((tank1, 'level'), above, tankhi, act_open330)
ctrl_close335 = ConditionalControl((tank1, 'level'), above, tankhi, act_close335)

wn.add_control(name='Fill_1Open_335', control_object=ctrl_open335)
wn.add_control(name='Fill_1Close330', control_object=ctrl_close330)
wn.add_control('Drain1Open_330', control_object=ctrl_open330)
wn.add_control('Drain1Close335', control_object=ctrl_close335)

### Add in the remaining elements
Now, the rest of the junctions, pipes, and time controls will be created without additional commentary. To save space,
the lines will not be formatted prettily, but extend off the screen.

In [ ]:
# Add the remaining junctions
wn.add_junction('20', elevation=toSI(flowunits, 129, hydPar.Elevation), coordinates=(29.44, 26.91))
wn.add_junction('35', elevation=toSI(flowunits, 12.5, hydPar.Elevation), base_demand=toSI(flowunits, 1, hydPar.Demand), demand_pattern='4', coordinates=(25.46, 10.52))
wn.add_junction('40', elevation=toSI(flowunits, 131.9, hydPar.Elevation), coordinates=(27.02, 9.81))
wn.add_junction('50', elevation=toSI(flowunits, 116.5, hydPar.Elevation), coordinates=(33.01, 3.01))
wn.add_junction('103', elevation=toSI(flowunits, 43, hydPar.Elevation), base_demand=toSI(flowunits, 133.2, hydPar.Demand), coordinates=(12.96, 21.31))
wn.add_junction('105', elevation=toSI(flowunits, 28.5, hydPar.Elevation), base_demand=toSI(flowunits, 135.37, hydPar.Demand), coordinates=(16.97, 21.28))
wn.add_junction('107', elevation=toSI(flowunits, 22, hydPar.Elevation), base_demand=toSI(flowunits, 54.64, hydPar.Demand), coordinates=(18.45, 20.46))
wn.add_junction('109', elevation=toSI(flowunits, 20.3, hydPar.Elevation), base_demand=toSI(flowunits, 231.4, hydPar.Demand), coordinates=(17.64, 18.92))
wn.add_junction('111', elevation=toSI(flowunits, 10, hydPar.Elevation), base_demand=toSI(flowunits, 141.94, hydPar.Demand), coordinates=(20.21, 17.53))
wn.add_junction('113', elevation=toSI(flowunits, 2, hydPar.Elevation), base_demand=toSI(flowunits, 20.01, hydPar.Demand), coordinates=(22.04, 16.61))
wn.add_junction('115', elevation=toSI(flowunits, 14, hydPar.Elevation), base_demand=toSI(flowunits, 52.1, hydPar.Demand), coordinates=(20.98, 19.18))
wn.add_junction('117', elevation=toSI(flowunits, 13.6, hydPar.Elevation), base_demand=toSI(flowunits, 117.71, hydPar.Demand), coordinates=(21.69, 21.28))
wn.add_junction('119', elevation=toSI(flowunits, 2, hydPar.Elevation), base_demand=toSI(flowunits, 176.13, hydPar.Demand), coordinates=(23.7, 22.76))
wn.add_junction('120', elevation=toSI(flowunits, 0, hydPar.Elevation), coordinates=(22.08, 23.1))
wn.add_junction('121', elevation=toSI(flowunits, -2, hydPar.Elevation), base_demand=toSI(flowunits, 41.63, hydPar.Demand), coordinates=(23.54, 25.5))
wn.add_junction('123', elevation=toSI(flowunits, 11, hydPar.Elevation), base_demand=toSI(flowunits, 1, hydPar.Demand), demand_pattern='2', coordinates=(23.37, 27.31))
wn.add_junction('125', elevation=toSI(flowunits, 11, hydPar.Elevation), base_demand=toSI(flowunits, 45.6, hydPar.Demand), coordinates=(24.59, 25.64))
wn.add_junction('127', elevation=toSI(flowunits, 56, hydPar.Elevation), base_demand=toSI(flowunits, 17.66, hydPar.Demand), coordinates=(29.29, 26.4))
wn.add_junction('129', elevation=toSI(flowunits, 51, hydPar.Elevation), coordinates=(30.32, 26.39))
wn.add_junction('131', elevation=toSI(flowunits, 6, hydPar.Elevation), base_demand=toSI(flowunits, 42.75, hydPar.Demand), coordinates=(37.89, 29.55))
wn.add_junction('139', elevation=toSI(flowunits, 31, hydPar.Elevation), base_demand=toSI(flowunits, 5.89, hydPar.Demand), coordinates=(33.28, 24.54))
wn.add_junction('141', elevation=toSI(flowunits, 4, hydPar.Elevation), base_demand=toSI(flowunits, 9.85, hydPar.Demand), coordinates=(35.68, 23.08))
wn.add_junction('143', elevation=toSI(flowunits, -4.5, hydPar.Elevation), base_demand=toSI(flowunits, 6.2, hydPar.Demand), coordinates=(37.47, 21.97))
wn.add_junction('145', elevation=toSI(flowunits, 1, hydPar.Elevation), base_demand=toSI(flowunits, 27.63, hydPar.Demand), coordinates=(33.02, 19.29))
wn.add_junction('147', elevation=toSI(flowunits, 18.5, hydPar.Elevation), base_demand=toSI(flowunits, 8.55, hydPar.Demand), coordinates=(30.24, 20.38))
wn.add_junction('149', elevation=toSI(flowunits, 16, hydPar.Elevation), base_demand=toSI(flowunits, 27.07, hydPar.Demand), coordinates=(29.62, 20.74))
wn.add_junction('151', elevation=toSI(flowunits, 33.5, hydPar.Elevation), base_demand=toSI(flowunits, 144.48, hydPar.Demand), coordinates=(28.29, 21.39))
wn.add_junction('153', elevation=toSI(flowunits, 66.2, hydPar.Elevation), base_demand=toSI(flowunits, 44.17, hydPar.Demand), coordinates=(28.13, 22.63))
wn.add_junction('157', elevation=toSI(flowunits, 13.1, hydPar.Elevation), base_demand=toSI(flowunits, 51.79, hydPar.Demand), coordinates=(24.85, 20.16))
wn.add_junction('159', elevation=toSI(flowunits, 6, hydPar.Elevation), base_demand=toSI(flowunits, 41.32, hydPar.Demand), coordinates=(23.12, 17.5))
wn.add_junction('161', elevation=toSI(flowunits, 4, hydPar.Elevation), base_demand=toSI(flowunits, 15.8, hydPar.Demand), coordinates=(25.1, 15.28))
wn.add_junction('163', elevation=toSI(flowunits, 5, hydPar.Elevation), base_demand=toSI(flowunits, 9.42, hydPar.Demand), coordinates=(25.39, 14.98))
wn.add_junction('164', elevation=toSI(flowunits, 5, hydPar.Elevation), coordinates=(25.98, 15.14))
wn.add_junction('166', elevation=toSI(flowunits, -2, hydPar.Elevation), base_demand=toSI(flowunits, 2.6, hydPar.Demand), coordinates=(26.48, 15.13))
wn.add_junction('167', elevation=toSI(flowunits, -5, hydPar.Elevation), base_demand=toSI(flowunits, 14.56, hydPar.Demand), coordinates=(25.88, 12.98))
wn.add_junction('169', elevation=toSI(flowunits, -5, hydPar.Elevation), coordinates=(25.68, 12.74))
wn.add_junction('171', elevation=toSI(flowunits, -4, hydPar.Elevation), base_demand=toSI(flowunits, 39.34, hydPar.Demand), coordinates=(26.65, 11.8))
wn.add_junction('173', elevation=toSI(flowunits, -4, hydPar.Elevation), coordinates=(26.87, 11.59))
wn.add_junction('177', elevation=toSI(flowunits, 8, hydPar.Elevation), base_demand=toSI(flowunits, 58.17, hydPar.Demand), coordinates=(25.71, 10.57))
wn.add_junction('179', elevation=toSI(flowunits, 8, hydPar.Elevation), coordinates=(25.71, 10.4))
wn.add_junction('181', elevation=toSI(flowunits, 8, hydPar.Elevation), coordinates=(25.72, 10.74))
wn.add_junction('183', elevation=toSI(flowunits, 11, hydPar.Elevation), coordinates=(25.45, 10.18))
wn.add_junction('184', elevation=toSI(flowunits, 16, hydPar.Elevation), coordinates=(25.15, 9.52))
wn.add_junction('185', elevation=toSI(flowunits, 16, hydPar.Elevation), base_demand=toSI(flowunits, 25.65, hydPar.Demand), coordinates=(25.01, 9.67))
wn.add_junction('187', elevation=toSI(flowunits, 12.5, hydPar.Elevation), coordinates=(23.64, 11.04))
wn.add_junction('189', elevation=toSI(flowunits, 4, hydPar.Elevation), base_demand=toSI(flowunits, 107.92, hydPar.Demand), coordinates=(24.15, 11.37))
wn.add_junction('191', elevation=toSI(flowunits, 25, hydPar.Elevation), base_demand=toSI(flowunits, 81.9, hydPar.Demand), coordinates=(22.1, 14.07))
wn.add_junction('193', elevation=toSI(flowunits, 18, hydPar.Elevation), base_demand=toSI(flowunits, 71.31, hydPar.Demand), coordinates=(22.88, 14.35))
wn.add_junction('195', elevation=toSI(flowunits, 15.5, hydPar.Elevation), coordinates=(23.18, 14.72))
wn.add_junction('197', elevation=toSI(flowunits, 23, hydPar.Elevation), base_demand=toSI(flowunits, 17.04, hydPar.Demand), coordinates=(20.97, 15.18))
wn.add_junction('199', elevation=toSI(flowunits, -2, hydPar.Elevation), base_demand=toSI(flowunits, 119.32, hydPar.Demand), coordinates=(29.42, 8.44))
wn.add_junction('201', elevation=toSI(flowunits, 0.1, hydPar.Elevation), base_demand=toSI(flowunits, 44.61, hydPar.Demand), coordinates=(30.89, 8.57))
wn.add_junction('203', elevation=toSI(flowunits, 2, hydPar.Elevation), base_demand=toSI(flowunits, 1, hydPar.Demand), demand_pattern='5', coordinates=(31.14, 8.89))
wn.add_junction('204', elevation=toSI(flowunits, 21, hydPar.Elevation), coordinates=(23.8, 10.9))
wn.add_junction('205', elevation=toSI(flowunits, 21, hydPar.Elevation), base_demand=toSI(flowunits, 65.36, hydPar.Demand), coordinates=(29.2, 6.46))
wn.add_junction('206', elevation=toSI(flowunits, 1, hydPar.Elevation), coordinates=(31.66, 6.64))
wn.add_junction('207', elevation=toSI(flowunits, 9, hydPar.Elevation), base_demand=toSI(flowunits, 69.39, hydPar.Demand), coordinates=(31, 6.61))
wn.add_junction('208', elevation=toSI(flowunits, 16, hydPar.Elevation), coordinates=(32.54, 6.81))
wn.add_junction('209', elevation=toSI(flowunits, -2, hydPar.Elevation), base_demand=toSI(flowunits, 0.87, hydPar.Demand), coordinates=(33.76, 6.59))
wn.add_junction('211', elevation=toSI(flowunits, 7, hydPar.Elevation), base_demand=toSI(flowunits, 8.67, hydPar.Demand), coordinates=(34.2, 5.54))
wn.add_junction('213', elevation=toSI(flowunits, 7, hydPar.Elevation), base_demand=toSI(flowunits, 13.94, hydPar.Demand), coordinates=(35.26, 6.16))
wn.add_junction('215', elevation=toSI(flowunits, 7, hydPar.Elevation), base_demand=toSI(flowunits, 92.19, hydPar.Demand), coordinates=(39.95, 8.73))
wn.add_junction('217', elevation=toSI(flowunits, 6, hydPar.Elevation), base_demand=toSI(flowunits, 24.22, hydPar.Demand), coordinates=(42.11, 8.67))
wn.add_junction('219', elevation=toSI(flowunits, 4, hydPar.Elevation), base_demand=toSI(flowunits, 41.32, hydPar.Demand), coordinates=(44.86, 9.32))
wn.add_junction('225', elevation=toSI(flowunits, 8, hydPar.Elevation), base_demand=toSI(flowunits, 22.8, hydPar.Demand), coordinates=(43.53, 7.38))
wn.add_junction('229', elevation=toSI(flowunits, 10.5, hydPar.Elevation), base_demand=toSI(flowunits, 64.18, hydPar.Demand), coordinates=(36.16, 3.49))
wn.add_junction('231', elevation=toSI(flowunits, 5, hydPar.Elevation), base_demand=toSI(flowunits, 16.48, hydPar.Demand), coordinates=(38.38, 2.54))
wn.add_junction('237', elevation=toSI(flowunits, 14, hydPar.Elevation), base_demand=toSI(flowunits, 15.61, hydPar.Demand), coordinates=(35.37, 3.08))
wn.add_junction('239', elevation=toSI(flowunits, 13, hydPar.Elevation), base_demand=toSI(flowunits, 44.61, hydPar.Demand), coordinates=(35.76, 2.31))
wn.add_junction('241', elevation=toSI(flowunits, 13, hydPar.Elevation), coordinates=(35.87, 2.11))
wn.add_junction('243', elevation=toSI(flowunits, 14, hydPar.Elevation), base_demand=toSI(flowunits, 4.34, hydPar.Demand), coordinates=(37.04, 0))
wn.add_junction('247', elevation=toSI(flowunits, 18, hydPar.Elevation), base_demand=toSI(flowunits, 70.38, hydPar.Demand), coordinates=(35.02, 2.05))
wn.add_junction('249', elevation=toSI(flowunits, 18, hydPar.Elevation), coordinates=(35.02, 1.81))
wn.add_junction('251', elevation=toSI(flowunits, 30, hydPar.Elevation), base_demand=toSI(flowunits, 24.16, hydPar.Demand), coordinates=(34.15, 1.1))
wn.add_junction('253', elevation=toSI(flowunits, 36, hydPar.Elevation), base_demand=toSI(flowunits, 54.52, hydPar.Demand), coordinates=(32.17, 1.88))
wn.add_junction('255', elevation=toSI(flowunits, 27, hydPar.Elevation), base_demand=toSI(flowunits, 40.39, hydPar.Demand), coordinates=(33.51, 2.45))
wn.add_junction('257', elevation=toSI(flowunits, 17, hydPar.Elevation), coordinates=(21.17, 23.32))
wn.add_junction('259', elevation=toSI(flowunits, 25, hydPar.Elevation), coordinates=(20.8, 23.4))
wn.add_junction('261', elevation=toSI(flowunits, 0, hydPar.Elevation), coordinates=(20.79, 21.45))
wn.add_junction('263', elevation=toSI(flowunits, 0, hydPar.Elevation), coordinates=(20.32, 21.57))
wn.add_junction('265', elevation=toSI(flowunits, 0, hydPar.Elevation), coordinates=(25.39, 13.6))
wn.add_junction('267', elevation=toSI(flowunits, 21, hydPar.Elevation), coordinates=(23.38, 12.95))
wn.add_junction('269', elevation=toSI(flowunits, 0, hydPar.Elevation), coordinates=(25.03, 12.14))
wn.add_junction('271', elevation=toSI(flowunits, 6, hydPar.Elevation), coordinates=(25.97, 11))
wn.add_junction('273', elevation=toSI(flowunits, 8, hydPar.Elevation), coordinates=(29.16, 7.38))
wn.add_junction('275', elevation=toSI(flowunits, 10, hydPar.Elevation), coordinates=(31.07, 8.29))
# Add the remaining pipes
wn.add_pipe('20', start_node_name='3', end_node_name='20', length=toSI(flowunits, 99, hydPar.Length), diameter=toSI(flowunits, 99, hydPar.PipeDiameter), roughness=toSI(flowunits, 199, hydPar.RoughnessCoeff))
wn.add_pipe('40', start_node_name='1', end_node_name='40', length=toSI(flowunits, 99, hydPar.Length), diameter=toSI(flowunits, 99, hydPar.PipeDiameter), roughness=toSI(flowunits, 199, hydPar.RoughnessCoeff))
wn.add_pipe('50', start_node_name='2', end_node_name='50', length=toSI(flowunits, 99, hydPar.Length), diameter=toSI(flowunits, 99, hydPar.PipeDiameter), roughness=toSI(flowunits, 199, hydPar.RoughnessCoeff))
wn.add_pipe('60', start_node_name='River', end_node_name='60', length=toSI(flowunits, 1231, hydPar.Length), diameter=toSI(flowunits, 24, hydPar.PipeDiameter), roughness=toSI(flowunits, 140, hydPar.RoughnessCoeff))
wn.add_pipe('101', start_node_name='10', end_node_name='101', length=toSI(flowunits, 14200, hydPar.Length), diameter=toSI(flowunits, 18, hydPar.PipeDiameter), roughness=toSI(flowunits, 110, hydPar.RoughnessCoeff))
wn.add_pipe('103', start_node_name='101', end_node_name='103', length=toSI(flowunits, 1350, hydPar.Length), diameter=toSI(flowunits, 16, hydPar.PipeDiameter), roughness=toSI(flowunits, 130, hydPar.RoughnessCoeff))
wn.add_pipe('105', start_node_name='101', end_node_name='105', length=toSI(flowunits, 2540, hydPar.Length), diameter=toSI(flowunits, 12, hydPar.PipeDiameter), roughness=toSI(flowunits, 130, hydPar.RoughnessCoeff))
wn.add_pipe('107', start_node_name='105', end_node_name='107', length=toSI(flowunits, 1470, hydPar.Length), diameter=toSI(flowunits, 12, hydPar.PipeDiameter), roughness=toSI(flowunits, 130, hydPar.RoughnessCoeff))
wn.add_pipe('109', start_node_name='103', end_node_name='109', length=toSI(flowunits, 3940, hydPar.Length), diameter=toSI(flowunits, 16, hydPar.PipeDiameter), roughness=toSI(flowunits, 130, hydPar.RoughnessCoeff))
wn.add_pipe('111', start_node_name='109', end_node_name='111', length=toSI(flowunits, 2000, hydPar.Length), diameter=toSI(flowunits, 12, hydPar.PipeDiameter), roughness=toSI(flowunits, 130, hydPar.RoughnessCoeff))
wn.add_pipe('112', start_node_name='115', end_node_name='111', length=toSI(flowunits, 1160, hydPar.Length), diameter=toSI(flowunits, 12, hydPar.PipeDiameter), roughness=toSI(flowunits, 130, hydPar.RoughnessCoeff))
wn.add_pipe('113', start_node_name='111', end_node_name='113', length=toSI(flowunits, 1680, hydPar.Length), diameter=toSI(flowunits, 12, hydPar.PipeDiameter), roughness=toSI(flowunits, 130, hydPar.RoughnessCoeff))
wn.add_pipe('114', start_node_name='115', end_node_name='113', length=toSI(flowunits, 2000, hydPar.Length), diameter=toSI(flowunits, 8, hydPar.PipeDiameter), roughness=toSI(flowunits, 130, hydPar.RoughnessCoeff))
wn.add_pipe('115', start_node_name='107', end_node_name='115', length=toSI(flowunits, 1950, hydPar.Length), diameter=toSI(flowunits, 8, hydPar.PipeDiameter), roughness=toSI(flowunits, 130, hydPar.RoughnessCoeff))
wn.add_pipe('116', start_node_name='113', end_node_name='193', length=toSI(flowunits, 1660, hydPar.Length), diameter=toSI(flowunits, 12, hydPar.PipeDiameter), roughness=toSI(flowunits, 130, hydPar.RoughnessCoeff))
wn.add_pipe('117', start_node_name='263', end_node_name='105', length=toSI(flowunits, 2725, hydPar.Length), diameter=toSI(flowunits, 12, hydPar.PipeDiameter), roughness=toSI(flowunits, 130, hydPar.RoughnessCoeff))
wn.add_pipe('119', start_node_name='115', end_node_name='117', length=toSI(flowunits, 2180, hydPar.Length), diameter=toSI(flowunits, 12, hydPar.PipeDiameter), roughness=toSI(flowunits, 130, hydPar.RoughnessCoeff))
wn.add_pipe('120', start_node_name='119', end_node_name='120', length=toSI(flowunits, 730, hydPar.Length), diameter=toSI(flowunits, 12, hydPar.PipeDiameter), roughness=toSI(flowunits, 130, hydPar.RoughnessCoeff))
wn.add_pipe('121', start_node_name='120', end_node_name='117', length=toSI(flowunits, 1870, hydPar.Length), diameter=toSI(flowunits, 12, hydPar.PipeDiameter), roughness=toSI(flowunits, 130, hydPar.RoughnessCoeff))
wn.add_pipe('122', start_node_name='121', end_node_name='120', length=toSI(flowunits, 2050, hydPar.Length), diameter=toSI(flowunits, 8, hydPar.PipeDiameter), roughness=toSI(flowunits, 130, hydPar.RoughnessCoeff))
wn.add_pipe('123', start_node_name='121', end_node_name='119', length=toSI(flowunits, 2000, hydPar.Length), diameter=toSI(flowunits, 30, hydPar.PipeDiameter), roughness=toSI(flowunits, 141, hydPar.RoughnessCoeff))
wn.add_pipe('125', start_node_name='123', end_node_name='121', length=toSI(flowunits, 1500, hydPar.Length), diameter=toSI(flowunits, 30, hydPar.PipeDiameter), roughness=toSI(flowunits, 141, hydPar.RoughnessCoeff))
wn.add_pipe('129', start_node_name='121', end_node_name='125', length=toSI(flowunits, 930, hydPar.Length), diameter=toSI(flowunits, 24, hydPar.PipeDiameter), roughness=toSI(flowunits, 130, hydPar.RoughnessCoeff))
wn.add_pipe('131', start_node_name='125', end_node_name='127', length=toSI(flowunits, 3240, hydPar.Length), diameter=toSI(flowunits, 24, hydPar.PipeDiameter), roughness=toSI(flowunits, 130, hydPar.RoughnessCoeff))
wn.add_pipe('133', start_node_name='20', end_node_name='127', length=toSI(flowunits, 785, hydPar.Length), diameter=toSI(flowunits, 20, hydPar.PipeDiameter), roughness=toSI(flowunits, 130, hydPar.RoughnessCoeff))
wn.add_pipe('135', start_node_name='127', end_node_name='129', length=toSI(flowunits, 900, hydPar.Length), diameter=toSI(flowunits, 24, hydPar.PipeDiameter), roughness=toSI(flowunits, 130, hydPar.RoughnessCoeff))
wn.add_pipe('137', start_node_name='129', end_node_name='131', length=toSI(flowunits, 6480, hydPar.Length), diameter=toSI(flowunits, 16, hydPar.PipeDiameter), roughness=toSI(flowunits, 130, hydPar.RoughnessCoeff))
wn.add_pipe('145', start_node_name='129', end_node_name='139', length=toSI(flowunits, 2750, hydPar.Length), diameter=toSI(flowunits, 8, hydPar.PipeDiameter), roughness=toSI(flowunits, 130, hydPar.RoughnessCoeff))
wn.add_pipe('147', start_node_name='139', end_node_name='141', length=toSI(flowunits, 2050, hydPar.Length), diameter=toSI(flowunits, 8, hydPar.PipeDiameter), roughness=toSI(flowunits, 130, hydPar.RoughnessCoeff))
wn.add_pipe('149', start_node_name='143', end_node_name='141', length=toSI(flowunits, 1400, hydPar.Length), diameter=toSI(flowunits, 8, hydPar.PipeDiameter), roughness=toSI(flowunits, 130, hydPar.RoughnessCoeff))
wn.add_pipe('151', start_node_name='15', end_node_name='143', length=toSI(flowunits, 1650, hydPar.Length), diameter=toSI(flowunits, 8, hydPar.PipeDiameter), roughness=toSI(flowunits, 130, hydPar.RoughnessCoeff))
wn.add_pipe('153', start_node_name='145', end_node_name='141', length=toSI(flowunits, 3510, hydPar.Length), diameter=toSI(flowunits, 12, hydPar.PipeDiameter), roughness=toSI(flowunits, 130, hydPar.RoughnessCoeff))
wn.add_pipe('155', start_node_name='147', end_node_name='145', length=toSI(flowunits, 2200, hydPar.Length), diameter=toSI(flowunits, 12, hydPar.PipeDiameter), roughness=toSI(flowunits, 130, hydPar.RoughnessCoeff))
wn.add_pipe('159', start_node_name='147', end_node_name='149', length=toSI(flowunits, 880, hydPar.Length), diameter=toSI(flowunits, 12, hydPar.PipeDiameter), roughness=toSI(flowunits, 130, hydPar.RoughnessCoeff))
wn.add_pipe('161', start_node_name='149', end_node_name='151', length=toSI(flowunits, 1020, hydPar.Length), diameter=toSI(flowunits, 8, hydPar.PipeDiameter), roughness=toSI(flowunits, 130, hydPar.RoughnessCoeff))
wn.add_pipe('163', start_node_name='151', end_node_name='153', length=toSI(flowunits, 1170, hydPar.Length), diameter=toSI(flowunits, 12, hydPar.PipeDiameter), roughness=toSI(flowunits, 130, hydPar.RoughnessCoeff))
wn.add_pipe('169', start_node_name='125', end_node_name='153', length=toSI(flowunits, 4560, hydPar.Length), diameter=toSI(flowunits, 8, hydPar.PipeDiameter), roughness=toSI(flowunits, 130, hydPar.RoughnessCoeff))
wn.add_pipe('171', start_node_name='119', end_node_name='151', length=toSI(flowunits, 3460, hydPar.Length), diameter=toSI(flowunits, 12, hydPar.PipeDiameter), roughness=toSI(flowunits, 130, hydPar.RoughnessCoeff))
wn.add_pipe('173', start_node_name='119', end_node_name='157', length=toSI(flowunits, 2080, hydPar.Length), diameter=toSI(flowunits, 30, hydPar.PipeDiameter), roughness=toSI(flowunits, 141, hydPar.RoughnessCoeff))
wn.add_pipe('175', start_node_name='157', end_node_name='159', length=toSI(flowunits, 2910, hydPar.Length), diameter=toSI(flowunits, 30, hydPar.PipeDiameter), roughness=toSI(flowunits, 141, hydPar.RoughnessCoeff))
wn.add_pipe('177', start_node_name='159', end_node_name='161', length=toSI(flowunits, 2000, hydPar.Length), diameter=toSI(flowunits, 30, hydPar.PipeDiameter), roughness=toSI(flowunits, 141, hydPar.RoughnessCoeff))
wn.add_pipe('179', start_node_name='161', end_node_name='163', length=toSI(flowunits, 430, hydPar.Length), diameter=toSI(flowunits, 30, hydPar.PipeDiameter), roughness=toSI(flowunits, 141, hydPar.RoughnessCoeff))
wn.add_pipe('180', start_node_name='163', end_node_name='164', length=toSI(flowunits, 150, hydPar.Length), diameter=toSI(flowunits, 14, hydPar.PipeDiameter), roughness=toSI(flowunits, 130, hydPar.RoughnessCoeff))
wn.add_pipe('181', start_node_name='164', end_node_name='166', length=toSI(flowunits, 490, hydPar.Length), diameter=toSI(flowunits, 14, hydPar.PipeDiameter), roughness=toSI(flowunits, 130, hydPar.RoughnessCoeff))
wn.add_pipe('183', start_node_name='265', end_node_name='169', length=toSI(flowunits, 590, hydPar.Length), diameter=toSI(flowunits, 30, hydPar.PipeDiameter), roughness=toSI(flowunits, 141, hydPar.RoughnessCoeff))
wn.add_pipe('185', start_node_name='167', end_node_name='169', length=toSI(flowunits, 60, hydPar.Length), diameter=toSI(flowunits, 8, hydPar.PipeDiameter), roughness=toSI(flowunits, 130, hydPar.RoughnessCoeff))
wn.add_pipe('186', start_node_name='187', end_node_name='204', length=toSI(flowunits, 99.9, hydPar.Length), diameter=toSI(flowunits, 8, hydPar.PipeDiameter), roughness=toSI(flowunits, 130, hydPar.RoughnessCoeff))
wn.add_pipe('187', start_node_name='169', end_node_name='171', length=toSI(flowunits, 1270, hydPar.Length), diameter=toSI(flowunits, 30, hydPar.PipeDiameter), roughness=toSI(flowunits, 141, hydPar.RoughnessCoeff))
wn.add_pipe('189', start_node_name='171', end_node_name='173', length=toSI(flowunits, 50, hydPar.Length), diameter=toSI(flowunits, 30, hydPar.PipeDiameter), roughness=toSI(flowunits, 141, hydPar.RoughnessCoeff))
wn.add_pipe('191', start_node_name='271', end_node_name='171', length=toSI(flowunits, 760, hydPar.Length), diameter=toSI(flowunits, 24, hydPar.PipeDiameter), roughness=toSI(flowunits, 130, hydPar.RoughnessCoeff))
wn.add_pipe('193', start_node_name='35', end_node_name='181', length=toSI(flowunits, 30, hydPar.Length), diameter=toSI(flowunits, 24, hydPar.PipeDiameter), roughness=toSI(flowunits, 130, hydPar.RoughnessCoeff))
wn.add_pipe('195', start_node_name='181', end_node_name='177', length=toSI(flowunits, 30, hydPar.Length), diameter=toSI(flowunits, 12, hydPar.PipeDiameter), roughness=toSI(flowunits, 130, hydPar.RoughnessCoeff))
wn.add_pipe('197', start_node_name='177', end_node_name='179', length=toSI(flowunits, 30, hydPar.Length), diameter=toSI(flowunits, 12, hydPar.PipeDiameter), roughness=toSI(flowunits, 130, hydPar.RoughnessCoeff))
wn.add_pipe('199', start_node_name='179', end_node_name='183', length=toSI(flowunits, 210, hydPar.Length), diameter=toSI(flowunits, 12, hydPar.PipeDiameter), roughness=toSI(flowunits, 130, hydPar.RoughnessCoeff))
wn.add_pipe('201', start_node_name='40', end_node_name='179', length=toSI(flowunits, 1190, hydPar.Length), diameter=toSI(flowunits, 12, hydPar.PipeDiameter), roughness=toSI(flowunits, 130, hydPar.RoughnessCoeff))
wn.add_pipe('202', start_node_name='185', end_node_name='184', length=toSI(flowunits, 99.9, hydPar.Length), diameter=toSI(flowunits, 8, hydPar.PipeDiameter), roughness=toSI(flowunits, 130, hydPar.RoughnessCoeff))
wn.add_pipe('203', start_node_name='183', end_node_name='185', length=toSI(flowunits, 510, hydPar.Length), diameter=toSI(flowunits, 8, hydPar.PipeDiameter), roughness=toSI(flowunits, 130, hydPar.RoughnessCoeff))
wn.add_pipe('204', start_node_name='184', end_node_name='205', length=toSI(flowunits, 4530, hydPar.Length), diameter=toSI(flowunits, 12, hydPar.PipeDiameter), roughness=toSI(flowunits, 130, hydPar.RoughnessCoeff))
wn.add_pipe('205', start_node_name='204', end_node_name='185', length=toSI(flowunits, 1325, hydPar.Length), diameter=toSI(flowunits, 12, hydPar.PipeDiameter), roughness=toSI(flowunits, 130, hydPar.RoughnessCoeff))
wn.add_pipe('207', start_node_name='189', end_node_name='183', length=toSI(flowunits, 1350, hydPar.Length), diameter=toSI(flowunits, 12, hydPar.PipeDiameter), roughness=toSI(flowunits, 130, hydPar.RoughnessCoeff))
wn.add_pipe('209', start_node_name='189', end_node_name='187', length=toSI(flowunits, 500, hydPar.Length), diameter=toSI(flowunits, 8, hydPar.PipeDiameter), roughness=toSI(flowunits, 130, hydPar.RoughnessCoeff))
wn.add_pipe('211', start_node_name='169', end_node_name='269', length=toSI(flowunits, 646, hydPar.Length), diameter=toSI(flowunits, 12, hydPar.PipeDiameter), roughness=toSI(flowunits, 130, hydPar.RoughnessCoeff))
wn.add_pipe('213', start_node_name='191', end_node_name='187', length=toSI(flowunits, 2560, hydPar.Length), diameter=toSI(flowunits, 12, hydPar.PipeDiameter), roughness=toSI(flowunits, 130, hydPar.RoughnessCoeff))
wn.add_pipe('215', start_node_name='267', end_node_name='189', length=toSI(flowunits, 1230, hydPar.Length), diameter=toSI(flowunits, 12, hydPar.PipeDiameter), roughness=toSI(flowunits, 130, hydPar.RoughnessCoeff))
wn.add_pipe('217', start_node_name='191', end_node_name='193', length=toSI(flowunits, 520, hydPar.Length), diameter=toSI(flowunits, 12, hydPar.PipeDiameter), roughness=toSI(flowunits, 130, hydPar.RoughnessCoeff))
wn.add_pipe('219', start_node_name='193', end_node_name='195', length=toSI(flowunits, 360, hydPar.Length), diameter=toSI(flowunits, 12, hydPar.PipeDiameter), roughness=toSI(flowunits, 130, hydPar.RoughnessCoeff))
wn.add_pipe('221', start_node_name='161', end_node_name='195', length=toSI(flowunits, 2300, hydPar.Length), diameter=toSI(flowunits, 8, hydPar.PipeDiameter), roughness=toSI(flowunits, 130, hydPar.RoughnessCoeff))
wn.add_pipe('223', start_node_name='197', end_node_name='191', length=toSI(flowunits, 1150, hydPar.Length), diameter=toSI(flowunits, 12, hydPar.PipeDiameter), roughness=toSI(flowunits, 130, hydPar.RoughnessCoeff))
wn.add_pipe('225', start_node_name='111', end_node_name='197', length=toSI(flowunits, 2790, hydPar.Length), diameter=toSI(flowunits, 12, hydPar.PipeDiameter), roughness=toSI(flowunits, 130, hydPar.RoughnessCoeff))
wn.add_pipe('229', start_node_name='173', end_node_name='199', length=toSI(flowunits, 4000, hydPar.Length), diameter=toSI(flowunits, 24, hydPar.PipeDiameter), roughness=toSI(flowunits, 141, hydPar.RoughnessCoeff))
wn.add_pipe('231', start_node_name='199', end_node_name='201', length=toSI(flowunits, 630, hydPar.Length), diameter=toSI(flowunits, 24, hydPar.PipeDiameter), roughness=toSI(flowunits, 141, hydPar.RoughnessCoeff))
wn.add_pipe('233', start_node_name='201', end_node_name='203', length=toSI(flowunits, 120, hydPar.Length), diameter=toSI(flowunits, 24, hydPar.PipeDiameter), roughness=toSI(flowunits, 130, hydPar.RoughnessCoeff))
wn.add_pipe('235', start_node_name='199', end_node_name='273', length=toSI(flowunits, 725, hydPar.Length), diameter=toSI(flowunits, 12, hydPar.PipeDiameter), roughness=toSI(flowunits, 130, hydPar.RoughnessCoeff))
wn.add_pipe('237', start_node_name='205', end_node_name='207', length=toSI(flowunits, 1200, hydPar.Length), diameter=toSI(flowunits, 12, hydPar.PipeDiameter), roughness=toSI(flowunits, 130, hydPar.RoughnessCoeff))
wn.add_pipe('238', start_node_name='207', end_node_name='206', length=toSI(flowunits, 450, hydPar.Length), diameter=toSI(flowunits, 12, hydPar.PipeDiameter), roughness=toSI(flowunits, 130, hydPar.RoughnessCoeff))
wn.add_pipe('239', start_node_name='275', end_node_name='207', length=toSI(flowunits, 1430, hydPar.Length), diameter=toSI(flowunits, 12, hydPar.PipeDiameter), roughness=toSI(flowunits, 130, hydPar.RoughnessCoeff))
wn.add_pipe('240', start_node_name='206', end_node_name='208', length=toSI(flowunits, 510, hydPar.Length), diameter=toSI(flowunits, 12, hydPar.PipeDiameter), roughness=toSI(flowunits, 130, hydPar.RoughnessCoeff))
wn.add_pipe('241', start_node_name='208', end_node_name='209', length=toSI(flowunits, 885, hydPar.Length), diameter=toSI(flowunits, 12, hydPar.PipeDiameter), roughness=toSI(flowunits, 130, hydPar.RoughnessCoeff))
wn.add_pipe('243', start_node_name='209', end_node_name='211', length=toSI(flowunits, 1210, hydPar.Length), diameter=toSI(flowunits, 16, hydPar.PipeDiameter), roughness=toSI(flowunits, 130, hydPar.RoughnessCoeff))
wn.add_pipe('245', start_node_name='211', end_node_name='213', length=toSI(flowunits, 990, hydPar.Length), diameter=toSI(flowunits, 16, hydPar.PipeDiameter), roughness=toSI(flowunits, 130, hydPar.RoughnessCoeff))
wn.add_pipe('247', start_node_name='213', end_node_name='215', length=toSI(flowunits, 4285, hydPar.Length), diameter=toSI(flowunits, 16, hydPar.PipeDiameter), roughness=toSI(flowunits, 130, hydPar.RoughnessCoeff))
wn.add_pipe('249', start_node_name='215', end_node_name='217', length=toSI(flowunits, 1660, hydPar.Length), diameter=toSI(flowunits, 16, hydPar.PipeDiameter), roughness=toSI(flowunits, 130, hydPar.RoughnessCoeff))
wn.add_pipe('251', start_node_name='217', end_node_name='219', length=toSI(flowunits, 2050, hydPar.Length), diameter=toSI(flowunits, 14, hydPar.PipeDiameter), roughness=toSI(flowunits, 130, hydPar.RoughnessCoeff))
wn.add_pipe('257', start_node_name='217', end_node_name='225', length=toSI(flowunits, 1560, hydPar.Length), diameter=toSI(flowunits, 12, hydPar.PipeDiameter), roughness=toSI(flowunits, 130, hydPar.RoughnessCoeff))
wn.add_pipe('261', start_node_name='213', end_node_name='229', length=toSI(flowunits, 2200, hydPar.Length), diameter=toSI(flowunits, 8, hydPar.PipeDiameter), roughness=toSI(flowunits, 130, hydPar.RoughnessCoeff))
wn.add_pipe('263', start_node_name='229', end_node_name='231', length=toSI(flowunits, 1960, hydPar.Length), diameter=toSI(flowunits, 12, hydPar.PipeDiameter), roughness=toSI(flowunits, 130, hydPar.RoughnessCoeff))
wn.add_pipe('269', start_node_name='211', end_node_name='237', length=toSI(flowunits, 2080, hydPar.Length), diameter=toSI(flowunits, 12, hydPar.PipeDiameter), roughness=toSI(flowunits, 130, hydPar.RoughnessCoeff))
wn.add_pipe('271', start_node_name='237', end_node_name='229', length=toSI(flowunits, 790, hydPar.Length), diameter=toSI(flowunits, 8, hydPar.PipeDiameter), roughness=toSI(flowunits, 130, hydPar.RoughnessCoeff))
wn.add_pipe('273', start_node_name='237', end_node_name='239', length=toSI(flowunits, 510, hydPar.Length), diameter=toSI(flowunits, 12, hydPar.PipeDiameter), roughness=toSI(flowunits, 130, hydPar.RoughnessCoeff))
wn.add_pipe('275', start_node_name='239', end_node_name='241', length=toSI(flowunits, 35, hydPar.Length), diameter=toSI(flowunits, 12, hydPar.PipeDiameter), roughness=toSI(flowunits, 130, hydPar.RoughnessCoeff))
wn.add_pipe('277', start_node_name='241', end_node_name='243', length=toSI(flowunits, 2200, hydPar.Length), diameter=toSI(flowunits, 12, hydPar.PipeDiameter), roughness=toSI(flowunits, 130, hydPar.RoughnessCoeff))
wn.add_pipe('281', start_node_name='241', end_node_name='247', length=toSI(flowunits, 445, hydPar.Length), diameter=toSI(flowunits, 10, hydPar.PipeDiameter), roughness=toSI(flowunits, 130, hydPar.RoughnessCoeff))
wn.add_pipe('283', start_node_name='239', end_node_name='249', length=toSI(flowunits, 430, hydPar.Length), diameter=toSI(flowunits, 12, hydPar.PipeDiameter), roughness=toSI(flowunits, 130, hydPar.RoughnessCoeff))
wn.add_pipe('285', start_node_name='247', end_node_name='249', length=toSI(flowunits, 10, hydPar.Length), diameter=toSI(flowunits, 12, hydPar.PipeDiameter), roughness=toSI(flowunits, 130, hydPar.RoughnessCoeff))
wn.add_pipe('287', start_node_name='247', end_node_name='255', length=toSI(flowunits, 1390, hydPar.Length), diameter=toSI(flowunits, 10, hydPar.PipeDiameter), roughness=toSI(flowunits, 130, hydPar.RoughnessCoeff))
wn.add_pipe('289', start_node_name='50', end_node_name='255', length=toSI(flowunits, 925, hydPar.Length), diameter=toSI(flowunits, 10, hydPar.PipeDiameter), roughness=toSI(flowunits, 130, hydPar.RoughnessCoeff))
wn.add_pipe('291', start_node_name='255', end_node_name='253', length=toSI(flowunits, 1100, hydPar.Length), diameter=toSI(flowunits, 10, hydPar.PipeDiameter), roughness=toSI(flowunits, 130, hydPar.RoughnessCoeff))
wn.add_pipe('293', start_node_name='255', end_node_name='251', length=toSI(flowunits, 1100, hydPar.Length), diameter=toSI(flowunits, 8, hydPar.PipeDiameter), roughness=toSI(flowunits, 130, hydPar.RoughnessCoeff))
wn.add_pipe('295', start_node_name='249', end_node_name='251', length=toSI(flowunits, 1450, hydPar.Length), diameter=toSI(flowunits, 12, hydPar.PipeDiameter), roughness=toSI(flowunits, 130, hydPar.RoughnessCoeff))
wn.add_pipe('297', start_node_name='120', end_node_name='257', length=toSI(flowunits, 645, hydPar.Length), diameter=toSI(flowunits, 8, hydPar.PipeDiameter), roughness=toSI(flowunits, 130, hydPar.RoughnessCoeff))
wn.add_pipe('299', start_node_name='257', end_node_name='259', length=toSI(flowunits, 350, hydPar.Length), diameter=toSI(flowunits, 8, hydPar.PipeDiameter), roughness=toSI(flowunits, 130, hydPar.RoughnessCoeff))
wn.add_pipe('301', start_node_name='259', end_node_name='263', length=toSI(flowunits, 1400, hydPar.Length), diameter=toSI(flowunits, 8, hydPar.PipeDiameter), roughness=toSI(flowunits, 130, hydPar.RoughnessCoeff))
wn.add_pipe('303', start_node_name='257', end_node_name='261', length=toSI(flowunits, 1400, hydPar.Length), diameter=toSI(flowunits, 8, hydPar.PipeDiameter), roughness=toSI(flowunits, 130, hydPar.RoughnessCoeff))
wn.add_pipe('305', start_node_name='117', end_node_name='261', length=toSI(flowunits, 645, hydPar.Length), diameter=toSI(flowunits, 12, hydPar.PipeDiameter), roughness=toSI(flowunits, 130, hydPar.RoughnessCoeff))
wn.add_pipe('307', start_node_name='261', end_node_name='263', length=toSI(flowunits, 350, hydPar.Length), diameter=toSI(flowunits, 12, hydPar.PipeDiameter), roughness=toSI(flowunits, 130, hydPar.RoughnessCoeff))
wn.add_pipe('309', start_node_name='265', end_node_name='267', length=toSI(flowunits, 1580, hydPar.Length), diameter=toSI(flowunits, 8, hydPar.PipeDiameter), roughness=toSI(flowunits, 130, hydPar.RoughnessCoeff))
wn.add_pipe('311', start_node_name='193', end_node_name='267', length=toSI(flowunits, 1170, hydPar.Length), diameter=toSI(flowunits, 12, hydPar.PipeDiameter), roughness=toSI(flowunits, 130, hydPar.RoughnessCoeff))
wn.add_pipe('313', start_node_name='269', end_node_name='189', length=toSI(flowunits, 646, hydPar.Length), diameter=toSI(flowunits, 12, hydPar.PipeDiameter), roughness=toSI(flowunits, 130, hydPar.RoughnessCoeff))
wn.add_pipe('315', start_node_name='181', end_node_name='271', length=toSI(flowunits, 260, hydPar.Length), diameter=toSI(flowunits, 24, hydPar.PipeDiameter), roughness=toSI(flowunits, 130, hydPar.RoughnessCoeff))
wn.add_pipe('317', start_node_name='273', end_node_name='275', length=toSI(flowunits, 2230, hydPar.Length), diameter=toSI(flowunits, 8, hydPar.PipeDiameter), roughness=toSI(flowunits, 130, hydPar.RoughnessCoeff))
wn.add_pipe('319', start_node_name='273', end_node_name='205', length=toSI(flowunits, 645, hydPar.Length), diameter=toSI(flowunits, 12, hydPar.PipeDiameter), roughness=toSI(flowunits, 130, hydPar.RoughnessCoeff))
wn.add_pipe('321', start_node_name='163', end_node_name='265', length=toSI(flowunits, 1200, hydPar.Length), diameter=toSI(flowunits, 30, hydPar.PipeDiameter), roughness=toSI(flowunits, 141, hydPar.RoughnessCoeff))
wn.add_pipe('323', start_node_name='201', end_node_name='275', length=toSI(flowunits, 300, hydPar.Length), diameter=toSI(flowunits, 12, hydPar.PipeDiameter), roughness=toSI(flowunits, 130, hydPar.RoughnessCoeff))
wn.add_pipe('325', start_node_name='269', end_node_name='271', length=toSI(flowunits, 1290, hydPar.Length), diameter=toSI(flowunits, 8, hydPar.PipeDiameter), roughness=toSI(flowunits, 130, hydPar.RoughnessCoeff))
wn.add_pipe('329', start_node_name='61', end_node_name='123', length=toSI(flowunits, 45500, hydPar.Length), diameter=toSI(flowunits, 30, hydPar.PipeDiameter), roughness=toSI(flowunits, 140, hydPar.RoughnessCoeff))
wn.add_pipe('333', start_node_name='601', end_node_name='61', length=toSI(flowunits, 1, hydPar.Length), diameter=toSI(flowunits, 30, hydPar.PipeDiameter), roughness=toSI(flowunits, 140, hydPar.RoughnessCoeff))
# Add the remaining controls
wn.add_control('Open_10At025', TimeControl(wn, 25*3600, 'SIM_TIME', False, act_open_pump10))
wn.add_control('Close10At039', TimeControl(wn, 39*3600, 'SIM_TIME', False, act_close_pump10))
wn.add_control('Open_10At049', TimeControl(wn, 49*3600, 'SIM_TIME', False, act_open_pump10))
wn.add_control('Close10At063', TimeControl(wn, 63*3600, 'SIM_TIME', False, act_close_pump10))
wn.add_control('Open_10At073', TimeControl(wn, 73*3600, 'SIM_TIME', False, act_open_pump10))
wn.add_control('Close10At087', TimeControl(wn, 87*3600, 'SIM_TIME', False, act_close_pump10))
wn.add_control('Open_10At097', TimeControl(wn, 97*3600, 'SIM_TIME', False, act_open_pump10))
wn.add_control('Close10At111', TimeControl(wn,111*3600, 'SIM_TIME', False, act_close_pump10))
wn.add_control('Open_10At121', TimeControl(wn,121*3600, 'SIM_TIME', False, act_open_pump10))
wn.add_control('Close10At135', TimeControl(wn,135*3600, 'SIM_TIME', False, act_close_pump10))
wn.add_control('Open_10At145', TimeControl(wn,145*3600, 'SIM_TIME', False, act_open_pump10))
wn.add_control('Close10At159', TimeControl(wn,159*3600, 'SIM_TIME', False, act_close_pump10))

## Summarize the model
WNTR provides a tool called `NetworkAnalyzer` that provides some simple ways to look
at the structure of the water network model. The outputs are in SI units -- i.e., meters (length), 
kilograms (mass), cubic-meters-per-second (volume), seconds (time)

In [ ]:
wntr.utils.tools.NetworkAnalyzer.describe(wn)

## Save the model
Finally, we will save the model for use in other demos. Once you save the model, you
can look at the INP file in a normal text editor, or by opening it in through jupyter.

In [ ]:
wn.write_inpfile('MyNewNet3.inp')